In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# read table

dataPayment = '../Datasets/Employee_Payroll.csv'
payroll = pd.read_csv(dataPayment)

# set default value to 0 for NaN numerical data
numeric_cols = payroll.select_dtypes(include=[np.number]).columns
payroll[numeric_cols] = payroll[numeric_cols].clip(lower=0)


payroll.fillna(0, inplace=True)
payroll['Office'] = payroll['Office'].astype(int)

# define column for 1/4 year discretization
payroll['Fiscal Quarter'] = payroll['Fiscal Quarter']*0.25 - 0.25
payroll['Fiscal Period'] = payroll['Fiscal Year'] + payroll['Fiscal Quarter']

pd.to_datetime(payroll['Original Hire Date'])

# parse hire date to get hire year
payroll['Original Hire Date'] = payroll['Original Hire Date'].str.split('/').str[2]
payroll['Original Hire Date'] = payroll['Original Hire Date'].astype(int)

_work_year = payroll["Fiscal Period"] - payroll['Original Hire Date']

In [ ]:
# specify table column included

payroll = payroll[['Fiscal Year', 'Fiscal Period', 'Job Code', 'Job Title', 'Base Pay', 'Position ID', 'Employee Identifier']]

# add Working Year Column
payroll["Working Year"] = _work_year

payroll.head(20)

In [ ]:
# aku gatau kenapa tapi ini error di tempatku pas buat cache jadi run yang bawahnya ini aja

arr = []

try:
    with open('../cache/ids.txt', "r") as f:
        for _id in f:
            arr.append(int(_id))
except:
    if not os.path.exists('../cache'):
        os.mkdir('../cache')
        
    _index = payroll['Job Code'].unique()
    for _id in _index:
        counts = len(payroll.groupby(['Job Code']).get_group(_id))
        if counts > 1000:
            arr.append(str(_id))
    with open('cache/ids.txt', 'w') as f:
        for _id in arr:
            f.write('%s\n' % _id)
    with open('cache/ids.txt', 'r') as f:
        arr = []
        for _id in f:
            arr.append(int(_id)) 
finally:
   f.close()

# :return : <List> arr : list of unique job id

In [ ]:
# jadi pake ini buat ngisi arr nya, tp ga ngebuat cache

arr = []

_index = payroll['Job Code'].unique()
for _id in _index:
    counts = len(payroll.groupby(['Job Code']).get_group(_id))
    if counts > 1000:
        arr.append(_id)



# :return : <List> arr : list of unique job id

In [ ]:
emps = []
max = 0
for _id in arr:
    df = payroll[payroll['Job Code'] == _id]
    emp_id = df['Employee Identifier'].unique()
    if len(emp_id) > 50:
        l = 75
    else:
        l = len(emp_id)

    for i in range(l):
        for i in range(100):
            _index = np.random.randint(0, l-1)
            if emp_id[_index] not in emps:
                emps.append(emp_id[_index])
                break


len(emps)

In [ ]:
df_id = payroll[payroll['Employee Identifier'].isin(emps)]
df_id

In [ ]:
up = df_id.drop(['Job Title', 'Employee Identifier'], axis=1)
X = up[['Job Code', 'Working Year', 'Position ID', 'Fiscal Period', 'Fiscal Year']]
y = up['Base Pay']

In [ ]:
correlation_matrix  = up.corr()
correlation_matrix['Base Pay']

In [ ]:
X = up.drop("Base Pay", axis=1).values
y = up["Base Pay"].values
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
y_test

In [ ]:
# KNN 15 menit

from scipy.spatial import distance
from collections import Counter

class KNN:
    def __init__(self, k):
        self.k = k
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
    
    def predict(self, X_test):
        final_output = []
        for i in range(len(X_test)):
            d = []
            votes = []
            for j in range(len(X_train)):
                dist = distance.euclidean(X_train[j] , X_test[i])
                d.append([dist, j])
            d.sort()
            d = d[0:self.k]
            for d, j in d:
                votes.append(y_train[j])
            ans = Counter(votes).most_common(1)[0][0]
            final_output.append(ans)
            
        return final_output
    
clf = KNN(3)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

In [ ]:
# COBA PAKE LIBRARY

from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors = 3)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
knn.score(X_test, y_test)

from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)
# from sklearn.metrics import mean_absolute_percentage_error
# mean_absolute_percentage_error(y_test, y_pred)

In [ ]:
# dari sini kebawah yg errornya 6000an :)

dist = []
for row in X_test:
    d = np.linalg.norm(X_train - row, axis = 1)
    dist.append(d)
dist = np.array(dist)
dist

In [ ]:
k = 3
nearest = np.argsort(dist, axis=1)
nearest = nearest[:, :k]
nearest

In [ ]:
y_nearest = y[nearest]
y_nearest

In [ ]:
y_pred = np.mean(y_nearest, axis=1)
y_pred
# import scipy
# y_pred = scipy.stats.mode(y_nearest, axis=1)[0].flatten()
# y_pred

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(y_test, y_pred)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

x = df_id['Job Code']
y = df_id['Working Year']
z = df_id['Base Pay']

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z,
           linewidths=1, alpha=.7,
           edgecolor='k',
           s = 200,
           c=z)
plt.show()